## Data Preparation notebook
Running this is not necessary to replicate the main resultsd & figures, as processed data is provided. This notebook is only there to document the operations used to assemble the raw data.

In [2]:
import numpy as np
import pandas as pd
import os
import re
import pickle
pd.options.mode.chained_assignment = None 

In [2]:
def import_data_and_sort(path):
    # print(path)
    df = pd.read_csv(path)
    # drop the suffix
    df["utr"] = df["utr"].str[:50]
    # reorder
    df.drop(['Unnamed: 0'], axis=1, inplace=True)  # drop first column
    if 'total_reads' in df:
        df.sort_values(by=['total_reads'], inplace=True, ascending=False)
    else:
        df.sort_values(by=['total'], inplace=True, ascending=False)
    df.reset_index(inplace=True, drop=True)  # necessary as sorting creates an extra index
    return df

def extract_fold_data(path):
    regex = re.compile(r".\d+.\d+")
    structures = []
    energies = []
    with open(path, 'r') as fh:
        i = 0
        for line in fh:
            if i % 2 != 0:
                structures.append(line.split(" ")[0])
                energies.append(float(regex.findall(line)[0]))
            i += 1
    return structures, energies

### We begin by reading in all the raw data, sorting it and removing suffixes not part of the UTR

In [3]:
""" Read in all Data """
path = "../Data/RawData/"
files = [os.path.join(path,file) for file in os.listdir(path) if file.startswith("GSM")]
df_list = {re.search("_(.*)\.",file).group(1):import_data_and_sort(file) for file in files}
print(df_list.keys())
# Remove the nonstandard chemistries
entriesToRemove = ['egfp_pseudo_1', 'egfp_pseudo_2', 'egfp_m1pseudo_1', 'egfp_m1pseudo_2']
for k in entriesToRemove:
    df_list.pop(k, None)
print(df_list.keys())

/data/ouga04b/ag_gagneur/home/karollus/.conda/envs/karollus-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (33,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


dict_keys(['egfp_unmod_1', 'egfp_pseudo_2', 'egfp_m1pseudo_1', 'egfp_m1pseudo_2', 'mcherry_1', 'mcherry_2', 'designed_library', 'egfp_unmod_2', 'egfp_pseudo_1'])
dict_keys(['egfp_unmod_1', 'mcherry_1', 'mcherry_2', 'designed_library', 'egfp_unmod_2'])


### We subset the data using the same cutoffs as in the sample code (except for the genetic algorithm data, where I impose a cutoff of minimum 200 reads (as I couldnt find which value is actually used)

In [4]:
# Subsetting the random mpra data
keys = ['egfp_unmod_1', 'egfp_unmod_2', 'mcherry_1', 'mcherry_2']
cuts = [280000, 300000, 180000, 170000]
for key, cutoff in zip(keys, cuts):
    df_list[key] = df_list[key].iloc[:cutoff].copy()

# Subsetting the human data    
human = df_list["designed_library"][(df_list["designed_library"]['library'] == 'human_utrs') | 
                                    (df_list["designed_library"]['library'] == 'snv')]
human = human.sort_values('total', ascending=False).reset_index(drop=True)
human = human.iloc[:25000].copy()

# Subsetting the genetic algorithm data
GA_types = ['step_random_to_best_allow_uatg',
 'step_random_to_best_no_uatgs',
 'step_worst_to_best_allow_uatg',
 'step_worst_to_best_no_uatg',
 'target_allow_uaug_allow_stop',
 'target_no_uaug_allow_stop',
 'target_no_uaug_no_stop']
GA = df_list['designed_library'][df_list['designed_library']["library"].isin(GA_types)]
GA = GA.iloc[:sum(GA["total"] >= 200)].copy()

df_list.pop("designed_library", None)
df_list["human"] = human
df_list["ga"] = GA

### We reduce to the needed columns (utr and rl) and add a library column

In [5]:
for key, df in df_list.items():
    df = df.filter(regex=("rl|utr"))
    df["library"] = key
    df_list[key] = df

### We prepare the test (20k), val (20k) and train (rest) split for all the sets except ga (only for training) and human (only for validation)

In [6]:
for key, df in df_list.items():
    df["set"] = ""
    if key == "human":
        df["set"] = "test"
    elif key == "ga":
        df["set"] = "train"
    else:
        df.loc[:20000, "set"] = "test"
        df.loc[20000:40000, "set"] = "val"
        df.loc[40000:, "set"] = "train"
    df_list[key] = df

### We combine the data into one large frame

In [7]:
combined_df = pd.concat(df_list.values())    
combined_df.reset_index(inplace=True, drop=True)

In [8]:
egfp_cds = "atgggcgaattaagtaagggcgaggagctgttcaccggggtg\
gtgcccatcctggtcgagctggacggcgacgtaaacggccacaagttcagcgtgtccggcgagggcgagggcgatgccacctacggca\
agctgaccctgaagttcatctgcaccaccggcaagctgcccgtgccctggcccaccctcgtgaccaccctgacctacggcgtgcagtgctt\
cagccgctaccccgaccacatgaagcagcacgacttcttcaagtccgccatgcccgaaggctacgtccaggagcgcaccatcttcttcaag\
gacgacggcaactacaagacccgcgccgaggtgaagttcgagggcgacaccctggtgaaccgcatcgagctgaagggcatcgacttca\
aggaggacggcaacatcctggggcacaagctggagtacaactacaacagccacaacgtctatatcatggccgacaagcagaagaacgg\
catcaaggtgaacttcaagatccgccacaacatcgaggacggcagcgtgcagctcgccgaccactaccagcagaacacccccatcggc\
gacggccccgtgctgctgcccgacaaccactacctgagcacccagtccaagctgagcaaagaccccaacgagaagcgcgatcacatgg\
tcctgctggagttcgtgaccgccgccgggatcactctcggcatggacgagctgtacaagttcgaataaagctag".upper()
egfp_3utr = "cgcctcgactgtgccttctagttgccagccatctgttgtttg".upper()
combined_df["cds"] = egfp_cds
combined_df["3utr"] = egfp_3utr

### We add the energy and the structure

In [9]:
struct, energy = extract_fold_data('../Data/Folding/mpra.energy')
combined_df["utr_struct"] = struct
combined_df["utr_energy"] = energy

### We collect the snv data

In [10]:
snv_df = pd.read_csv("../Data/SNV/snv_phenotype_log_diff.csv")
snv_df.drop(['Unnamed: 0'], axis=1, inplace=True)  # drop first column
snv_df = snv_df[snv_df['obs_diff'] != 0.0]
snv_df = snv_df[snv_df['total'] >= 620]

In [11]:
snv_df["cds"] = egfp_cds
snv_df["3utr"] = egfp_3utr

### We collect the PTR data

In [12]:
# We get the PTR data
ptr_df = pd.read_csv("../Data/PTR/ptr.tsv", sep='\t')
# We average over all tissues
ptr_vals = ptr_df.select(lambda col: col.endswith('PTR'), axis=1).apply(pd.to_numeric, errors='coerce').median(axis=1)
ptr_df["ptr"] = ptr_vals

# We get the sequences
seq_df = pd.read_csv("../Data/PTR/seq.tsv", sep='\t')

# We combine
combined_df_ptr = seq_df[["GeneName","UTR5_Sequence", "CDS_Sequence", "UTR3_Sequence"]].merge(ptr_df)
combined_df_ptr = combined_df_ptr.rename(index=str, columns={"UTR5_Sequence": "utr", 
                                                            "CDS_Sequence":"cds",
                                                            "UTR3_Sequence":"3utr"})

/data/ouga04b/ag_gagneur/home/karollus/.conda/envs/karollus-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: 'select' is deprecated and will be removed in a future release. You can use .loc[labels.map(crit)] as a replacement
  after removing the cwd from sys.path.


### Collect the Riboseq data, compute the load and combine it with sequence data

In [21]:
# We get the sequences
seq_df = pd.read_csv("../Data/PTR/seq.tsv", sep='\t')
seq_df = seq_df.rename(index=str, columns={"UTR5_Sequence": "utr",
                                          "CDS_Sequence":"cds",
                                          "UTR3_Sequence":"3utr"})

#We get the andreev riboseq data and combine
andreev_df = pd.read_csv("../Data/RiboSeq/andreev_counts.tsv", sep='\t', decimal=",")
andreev_df = andreev_df.rename(index=str, columns={"Gene name": "GeneName", 
                                                   "Riboseq control reads, coding": "rpf",
                                                   "RNAseq control, (normalised)": "rnaseq_norm"})

andreev_df = andreev_df[(andreev_df["rpf"] > 10) & (andreev_df["rnaseq_norm"] > 10)]
andreev_df["log_load"] = np.log(andreev_df["rpf"]/andreev_df["rnaseq_norm"])

andreev_merged = andreev_df.merge(seq_df)

In [14]:
#We get the xtail pcr3 riboseq data and combine
pcr3_df = pd.read_csv("../Data/RiboSeq/xtail_counts_pcr3.tsv", sep='\t', decimal=",")
pcr3_df = pcr3_df.rename(index=str, columns={"Ensembl_ID": "EnsemblGeneID"})
pcr3_df = pcr3_df.rename(columns=lambda x: re.sub('.1$','_normalized',x))

pcr3_df["rpf"] = (pcr3_df["control1(RPF)_normalized"] + pcr3_df["control2(RPF)_normalized"])/2
pcr3_df["rna"] = (pcr3_df["control1(mRNA)_normalized"] + pcr3_df["control2(mRNA)_normalized"])/2
pcr3_df = pcr3_df[(pcr3_df["rpf"] > 10) & (pcr3_df["rna"] > 10)]
pcr3_df["log_load"] = np.log(pcr3_df["rpf"]/pcr3_df["rna"])

pcr3_merged = pcr3_df.merge(seq_df)

In [15]:
#We get the Eichhorn hek293 riboseq data and combine
eichhorn_df = pd.read_csv("../Data/RiboSeq/Eichhorn_GSE60426_MockHEK293T.tsv", sep='\t', decimal=".")

eichhorn_df["log_load"] = np.log(eichhorn_df["RPF_RPKM"]/eichhorn_df["RNA_RPKM"])

eichhorn_merged = eichhorn_df.merge(seq_df)
eichhorn_merged = eichhorn_merged.dropna()

### Prepare more PTR data

In [16]:
# We get the wilhelm PTR data
wilhelm_ptr_df = pd.read_csv("../Data/PTR/wilhelm_ptr.tsv", sep='\t', decimal=",")
wilhelm_ptr_df = wilhelm_ptr_df.dropna()

wilhelm_ptr_df = wilhelm_ptr_df.rename(index=str, columns={"Accessions": "EnsemblGeneID",
                                                          "protein/mRNA ratio": "ptr"})
combined_wilhelm = seq_df.merge(wilhelm_ptr_df, on="EnsemblGeneID")

### Add the polysome profiling data

In [17]:
doudna_df = pd.read_csv("../Data/TrIP-Seq/doudna_polysome_tripseq_isoform_tpm_ensembl_v75.csv")
doudna_df = doudna_df.rename(index=str, columns={"gene_id": "EnsemblGeneID",
                                                 "isoform_id": "EnsemblTranscriptID",
                                                      "gene_name": "GeneName"})


# replicate 1
fractions_1 = doudna_df.select(lambda col: re.match("poly._1|80S_1|cyto_1", col), axis=1)
doudna_df["count_1"] = fractions_1.sum(axis=1)
doudna_df["rl_1"] = np.sum(np.array(fractions_1) * np.arange(0,9), axis=1)/np.sum(np.array(fractions_1),axis=1)
# replicate 2
fractions_2 = doudna_df.select(lambda col: re.match("poly._2|80S_2|cyto_2", col), axis=1)
doudna_df["count_2"] = fractions_2.sum(axis=1)
doudna_df["rl_2"] = np.sum(np.array(fractions_2) * np.arange(0,9), axis=1)/np.sum(np.array(fractions_2),axis=1)
# replicate mean
fractions = (np.array(fractions_1) + np.array(fractions_2))/2
doudna_df["count_mean"] = np.sum(fractions, axis=1)
doudna_df["rl_mean"] = np.sum(fractions * np.arange(0,9), axis=1)/np.sum(fractions,axis=1)

/data/ouga04b/ag_gagneur/home/karollus/.conda/envs/karollus-env/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: 'select' is deprecated and will be removed in a future release. You can use .loc[labels.map(crit)] as a replacement
  
/data/ouga04b/ag_gagneur/home/karollus/.conda/envs/karollus-env/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
/data/ouga04b/ag_gagneur/home/karollus/.conda/envs/karollus-env/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: 'select' is deprecated and will be removed in a future release. You can use .loc[labels.map(crit)] as a replacement
  if sys.path[0] == '':
/data/ouga04b/ag_gagneur/home/karollus/.conda/envs/karollus-env/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  
/data/ouga04b/ag_gagneur/home/karollus/.conda/envs/karollus-env/lib/p

In [18]:
seq_df = pd.read_csv("../Data/gencodev19_seq.csv")
combined_doudna = seq_df.merge(doudna_df, on="EnsemblTranscriptID")

#### Combine transcripts with same 5utr

In [19]:
doudna_rep1 = combined_doudna[(combined_doudna["count_1"] > 1)]
doudna_rep2 = combined_doudna[(combined_doudna["count_2"] > 1)]
# replicate 1
fractions_1 = doudna_rep1.select(lambda col: re.match("poly._1|80S_1|cyto_1|utr", col), axis=1).groupby("utr").sum()
count_1 = fractions_1.sum(axis=1)
rl_1 = np.sum(np.array(fractions_1) * np.arange(0,9), axis=1)/np.sum(np.array(fractions_1),axis=1)
aggreg_1 = pd.DataFrame({"utr":fractions_1.index,"count_1":list(count_1),"rl_1":rl_1})
# replicate 2
fractions_2 = doudna_rep2.select(lambda col: re.match("poly._2|80S_2|cyto_2|utr", col), axis=1).groupby("utr").sum()
count_2 = fractions_2.sum(axis=1)
rl_2 = np.sum(np.array(fractions_2) * np.arange(0,9), axis=1)/np.sum(np.array(fractions_2),axis=1)
aggreg_2 = pd.DataFrame({"utr":fractions_2.index,"count_2":list(count_2),"rl_2":rl_2})
# merge
aggreg = aggreg_1.merge(aggreg_2, on="utr")
# mean
aggreg["rl_mean"] = (aggreg["count_1"]*aggreg["rl_1"] + aggreg["count_2"]*aggreg["rl_2"])/(aggreg["count_1"] + aggreg["count_2"])

/data/ouga04b/ag_gagneur/home/karollus/.conda/envs/karollus-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: 'select' is deprecated and will be removed in a future release. You can use .loc[labels.map(crit)] as a replacement
  after removing the cwd from sys.path.
/data/ouga04b/ag_gagneur/home/karollus/.conda/envs/karollus-env/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: 'select' is deprecated and will be removed in a future release. You can use .loc[labels.map(crit)] as a replacement
  if __name__ == '__main__':


### Add all the data to a dict and pickle

In [22]:
data_dict = {"mpra":combined_df, "snv":snv_df, 
             "ptr":combined_df_ptr, "wilhelm": combined_wilhelm,
             "andreev":andreev_merged, "pcr3":pcr3_merged, "eichhorn": eichhorn_merged,
            "doudna":aggreg}
with open("../Data/data_dict.pkl", 'wb') as handle:
    pickle.dump(data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Add extended data

In [45]:
with open("../Data/data_dict.pkl", 'rb') as handle:
    data_dict = pickle.load(handle)
varlen_df = pd.read_pickle("../Data/RawData/varying_length_25to100_1.pkl")

In [46]:
varlen_df["library"] = varlen_df["set"]
## Filter out UTRs with too few less reads
varlen_df = varlen_df[varlen_df['total_reads']>=10]
print(len(varlen_df[varlen_df['set']=='random']))
print(len(varlen_df[varlen_df['set']=='human']))
varlen_df.sort_values(['len', 'total_reads'], inplace=True, ascending=False)
varlen_df.reset_index(inplace=True, drop=True)

83919
15555


In [47]:
varlen_df["set"] = "train"
for i in range(25,101):
    idx = varlen_df[(varlen_df['len']==i) & (varlen_df['library']=="random")].iloc[:100].index
    varlen_df.loc[idx, "set"] = "test"
    idx_human = varlen_df[(varlen_df['len']==i) & (varlen_df['library']=="human")].iloc[:100].index
    varlen_df.loc[idx_human, "set"] = "test"
print(len(varlen_df[(varlen_df['library']=="random") & (varlen_df['set']=="test")]))
print(len(varlen_df[(varlen_df['library']=="human") & (varlen_df['set']=="test")]))

7600
7600


In [52]:
# check that there is no intersection of training and test set
set(varlen_df[(varlen_df.set == "train") & (varlen_df['library']=="random")]["utr"]) & set(varlen_df[(varlen_df.set == "test") & (varlen_df['library']=="random")]["utr"]) 

set()

In [54]:
# Add to data dict and pickle
data_dict["varlen_mpra"] = varlen_df

In [56]:
with open("../Data/data_dict.pkl", 'wb') as handle:
    pickle.dump(data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)